In [1]:
import sys
sys.path.append('../')  # Füge das übergeordnete Verzeichnis zum Python-Pfad hinzu

# Jetzt kannst du das Modul aus dem übergeordneten Verzeichnis importieren
from gold_standard import * 

In [7]:
# Read XML-Files
df_eafc = pd.read_xml('../../Football-Project/data/xml-files/eafc_final.xml')
df_fm23 = pd.read_xml('../../Football-Project/data/xml-files/fm23_final.xml')
# df_tm = pd.read_xml('../../Football-Project/data/xml-files/tm_final.xml')

# Take only a sample of the whole dataset. Saves compute power
subset_size = 2000
df_fm23_sampled = df_fm23.sample(n=subset_size, random_state=42)

In [8]:
# Create similarity matrixes for differenct similarity functions and columns
similarity_matrix_name = get_similarity_matrix(leventstein_similarity, df_fm23_sampled['name'], df_eafc['name'])
similarity_matrix_date = get_similarity_matrix(date_similarity, df_fm23_sampled['birthdate'], df_eafc['birthdate'])
#similarity_matrix_club = get_similarity_matrix(leventstein_similarity, df_fm23_sampled['club'], df_eafc['club'])

In [9]:
# df_name = create_result(df_fm23_sampled, df_eafc, similarity_matrix_name)
# df_date = create_result(df_fm23_sampled, df_eafc, similarity_matrix_date)

# Create the result dataframe and weight the importances of similarities.
# Name has a very high probability, because there are a lot name abbreviations in the data resulting in lower similarity scores
df = create_result(df_fm23_sampled, df_eafc, (1.3*similarity_matrix_name) + (0.7*similarity_matrix_date), 2).sort_values(by=['score'], ascending=False)

# Plot histogram to see the distribution of the similarity scores.
#df.score.hist()

### Figure out suitable values for perfect_match_at_least_threshhold and corner_case_at_least_threshhold

In [10]:
# pd.set_option('display.max_rows', None)
display(df)

,df1,df2,name1,birthdate1,name2,birthdate2,nat1,nat2,club1,club2,score
9366,fm_10366,eafc_11500,Yang Shuai,1997-01-28,Yang Shuai,1997-01-28,CHN,China PR,Henan,Henan,1.000000
15226,fm_16226,eafc_8450,Ekain,1994-04-29,Ekain,1994-04-29,ESP,Spain,Ibiza,Racing Santander,1.000000
9247,fm_10247,eafc_11544,Zhu Chenjie,2000-08-23,Zhu Chenjie,2000-08-23,CHN,China PR,SH Shenhua,Shanghai Shenhua,1.000000
13252,fm_14252,eafc_2695,João Mário,1993-01-19,João Mário,1993-01-19,POR,Portugal,Benfica,Benfica,1.000000
5990,fm_6990,eafc_7692,Leandro Sanca,2000-01-04,Leandro Sanca,2000-01-04,POR,Portugal,Spezia,Chaves,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
16286,fm_17286,eafc_9574,Hadji Kebe,2006-06-15,Samuel Soares,2002-06-15,FRA,Portugal,QRM,Benfica,0.325000
2129,fm_3129,eafc_1092,Mike Könnecke,1988-08-23,M. Mancosu,1988-08-22,GER,Italy,FSV Zwickau,Cagliari,0.325000
16379,fm_17379,eafc_1911,Dylan Rustico,2006-03-25,D. Bisoli,1994-03-25,FRA,Italy,Strasbourg,Brescia,0.325000
11813,fm_12813,eafc_8988,Kelechi Chibueze,2003-12-14,A. Al Ayeri,2003-12-12,ENG,Saudi Arabia,Leicester,Al Taawon,0.314583


In [11]:
# create datasets for perfect_matches, corner_cases, non_matches
perfect_match_at_least_threshhold = 1
corner_case_at_least_threshhold = 0.5

perfect_matches = get_perfect_matches(df, perfect_match_at_least_threshhold)
corner_cases = get_corner_cases(df, perfect_match_at_least_threshhold, corner_case_at_least_threshhold)
non_matches = get_non_matches(df, corner_case_at_least_threshhold)

print(f"Length of perfect_matches: {len(perfect_matches)}, {(len(perfect_matches) / len(df) * 100):.2f}%")
print(f"Length of corner cases: {len(corner_cases)}, {(len(corner_cases) / len(df) * 100):.2f}%")
print(f"Length of non_matches {len(non_matches)}, {(len(non_matches) / len(df) * 100):.2f}%")

Length of perfect_matches: 128, 6.40%
Length of corner cases: 904, 45.20%
Length of non_matches 968, 48.40%


In [12]:
# Save all perfect matches
perfect_matches.to_csv('perfect_matches_fm_ea_v2.csv', index=False)

# Save corner cases to disk
# max_corner_cases = None
# corner_cases.sort_values(by=['score'], ascending=True).head(max_corner_cases).to_csv('corner_cases_fm_ea_v2.csv', index=False)

# # Save non matches to disk
# non_matches.to_csv('non_matches_fm_ea_v2.csv', index=False)